In [4]:
import gym
import tensorflow as tf
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from rl_utils import load_expert_policy, SupervisedPolicy, Dagger, run_rollouts, get_valdata, render_demo, render_NOerrors

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display
    
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=25)
    display(display_animation(anim, default_mode='loop'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Avialable envs:
- RoboschoolHumanoid_v0_2017may
- RoboschoolHalfCheetah_v0_2017may
- RoboschoolHumanoidFlagrunHarder_v0_2017may
- RoboschoolHopper_v0_2017may

In [11]:
# Loading the expert
env, expert, env_name = load_expert_policy('RoboschoolHumanoid_v0_2017may')

# Get some data
observations, actions, rewards = run_rollouts(env, expert, env.spec.timestep_limit, 20)
train_data = (observations, actions)

val_data = get_valdata(env, expert, env.spec.timestep_limit)

loaded :3
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [12]:
# Loading a clonner
noob = SupervisedPolicy(env.observation_space, env.action_space)
noob.train(train_data, val_data, 300, verbose=0)

In [ ]:
noob_frames = render_NOerrors(env_name, noob, steps=300)
display_frames_as_gif(noob_frames)

DAgger
------------
Now use the Data aggregation algorithm from, gonna use the same environment

In [14]:
noob2 = SupervisedPolicy(env.observation_space, env.action_space)
dagger = Dagger(env, expert, noob2, 50000)

In [15]:
dagger.train(env.spec.timestep_limit, verbose=0, iterations=200)

DAgger iter 0
DAgger iter 1
DAgger iter 2
DAgger iter 3
DAgger iter 4
DAgger iter 5
DAgger iter 6
DAgger iter 7
DAgger iter 8
DAgger iter 9
DAgger iter 10
DAgger iter 11
DAgger iter 12
DAgger iter 13
DAgger iter 14
DAgger iter 15
DAgger iter 16
DAgger iter 17
DAgger iter 18
DAgger iter 19
DAgger iter 20
DAgger iter 21
DAgger iter 22
DAgger iter 23
DAgger iter 24
DAgger iter 25
DAgger iter 26
DAgger iter 27
DAgger iter 28
DAgger iter 29
DAgger iter 30
DAgger iter 31
DAgger iter 32
DAgger iter 33
DAgger iter 34
DAgger iter 35
DAgger iter 36
DAgger iter 37
DAgger iter 38
DAgger iter 39
DAgger iter 40
DAgger iter 41
DAgger iter 42
DAgger iter 43
DAgger iter 44
DAgger iter 45
DAgger iter 46
DAgger iter 47
DAgger iter 48
DAgger iter 49
DAgger iter 50
DAgger iter 51
DAgger iter 52
DAgger iter 53
DAgger iter 54
DAgger iter 55
DAgger iter 56
DAgger iter 57
DAgger iter 58
DAgger iter 59
DAgger iter 60
DAgger iter 61
DAgger iter 62
DAgger iter 63
DAgger iter 64
DAgger iter 65
DAgger iter 66
DAgge

In [ ]:
noob2_frames = render_NOerrors(env_name, noob2, steps=300)
display_frames_as_gif(noob2_frames)

In [ ]:
noob.save_json()

In [ ]:
frames = render_demo(env, noob, 300)
display_frames_as_gif(frames)

In [ ]:
noob.load_json()